<a href="https://colab.research.google.com/github/supriyag123/PHD_Pub/blob/main/DyFor%20GP%20Heuristics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import math
import plotly.graph_objects as go
import keras
from tensorflow import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, RepeatVector, TimeDistributed, Input
from keras.models import Model
from keras import saving

import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import EarlyStopping
from keras.callbacks import Callback
import plotly
from keras import losses
import plotly.express as px # for data visualization
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
import numpy as np
import scipy.stats as stats
import pylab as pl



##### Take one dataset at a time
#train_data = np.load(r'/content/drive/MyDrive/PHD/2024/TEMP_OUTPUT_METROPM/multivariate_long_sequences-TRAIN-Daily-DIRECT-VAR.npy')
#train_data = np.load(r'/content/drive/MyDrive/PHD/2024/TEMP_OUTPUT_AIRQUALITY/multivariate_long_sequences-TRAIN-Daily-DIRECT-VAR.npy')
train_data = np.load(r'/content/drive/MyDrive/PHD/2024/TEMP_OUTPUT_ELECTRICITY/multivariate_long_sequences-TRAIN-DATA-FullSearch-VAR.npy')


n_seq = train_data.shape[0]
window_size = train_data.shape[1]
n_features = train_data.shape[2]
n_future = n_features
##Get large set of subsequence from previous steps (VAE)... this is not labelled yet.
#results1=np.load(r'/content/drive/MyDrive/PHD/2024/TEMP_OUTPUT_METROPM/generated_large_subsquence2_data_V2.npy')
#results1=np.load(r'/content/drive/MyDrive/PHD/2024/TEMP_OUTPUT_AIRQUALITY/generated_large_subsquence2_data.npy')
results1=np.load(r'/content/drive/MyDrive/PHD/2024/TEMP_OUTPUT_ELECTRICITY/generated_large_subsquence2_data.npy')
x=results1



##############DyFOR GP Heuristic#######################################################
from statsmodels.tsa.arima.model import ARIMA

def get_window_size (series, pred_point, small_w, large_w):
  # pred_point ==> the point of prediction (starting point)
  # small_w ==> smaller window size preceding the pred_point
  # large_w ==> larger window size preceding the pred_point
  # series is flttened long subsequence --> multivariate converted into univariate by flattening...


  pred_l_start = pred_point - large_w
  pred_s_start = pred_point - small_w
  pred_l_window = series[pred_l_start:pred_point]
  pred_s_window = series[pred_s_start:pred_point]
  modell = ARIMA(pred_l_window, order=(2,0,0))
  fitl = modell.fit()
  forecastl = fitl.forecast(steps=n_future)
  models = ARIMA(pred_s_window, order=(2,0,0))
  fits = models.fit()
  forecasts = fits.forecast(steps=n_future)
  mse_l = mean_squared_error(series[pred_point:pred_point+n_future], forecastl)
  mse_s = mean_squared_error(series[pred_point:pred_point+n_future], forecasts)
  if mse_l < mse_s:
    return large_w
  else:
    return small_w


best_window_for_long_seq = list()
best_window_for_long_seq.clear()



for i in range(x.shape[0]):
  print("starting sequence: ", i, " out of ", x.shape[0])
  cur_seq = x[i,:]
  start_small_w = 2
  start_large_w = 3
  start_index = start_large_w
  ### we need to increment by number of variables, as we have flattened the ime series
  for k in range (start_index*n_features, x.shape[1]-n_features+1, n_features):
    print("starting point: ", k, " out of ", x.shape[1]-n_features)
    pred_point_k = k
    small_w_k = start_small_w*n_features
    large_w_k = start_large_w*n_features
    print("small_w_k: ", small_w_k, " large_w_k: ", large_w_k)
    best_window_k = get_window_size(cur_seq, pred_point_k, small_w_k, large_w_k)
    print("best_window_k: ", best_window_k)
    if best_window_k == large_w_k:
      start_large_w = start_large_w + 1
      start_small_w = start_small_w + 1
    if best_window_k == small_w_k:
      if start_small_w > 1:
        start_large_w = start_large_w -1
        start_small_w = start_small_w -1
      else:
        start_large_w = start_large_w
        start_small_w = start_small_w
    print("getting best window for this pred_point",  k)
    final_best_window =  get_window_size(cur_seq, k, small_w_k, large_w_k)
    print("final_best_window: ", final_best_window)
    best_window_for_long_seq.append(final_best_window)


print(best_window_for_long_seq)
Window = np.array(best_window_for_long_seq)
np.save(r'/content/drive/MyDrive/PHD/2024/TEMP_OUTPUT_ELECTRICITY/generated-data-DyFOR_Window',Window)


starting sequence:  0  out of  136300
starting point:  18  out of  174
small_w_k:  12  large_w_k:  18
best_window_k:  18
getting best window for this pred_point 18
final_best_window:  18
starting point:  24  out of  174
small_w_k:  18  large_w_k:  24
best_window_k:  24
getting best window for this pred_point 24
final_best_window:  24
starting point:  30  out of  174
small_w_k:  24  large_w_k:  30
best_window_k:  30
getting best window for this pred_point 30
final_best_window:  30
starting point:  36  out of  174
small_w_k:  30  large_w_k:  36
best_window_k:  30
getting best window for this pred_point 36
final_best_window:  30
starting point:  42  out of  174
small_w_k:  24  large_w_k:  30
best_window_k:  24
getting best window for this pred_point 42
final_best_window:  24
starting point:  48  out of  174
small_w_k:  18  large_w_k:  24
best_window_k:  24
getting best window for this pred_point 48
final_best_window:  24
starting point:  54  out of  174
small_w_k:  24  large_w_k:  30
best

/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


best_window_k:  24
getting best window for this pred_point 42
final_best_window:  24
starting point:  48  out of  174
small_w_k:  18  large_w_k:  24
best_window_k:  24
getting best window for this pred_point 48
final_best_window:  24
starting point:  54  out of  174
small_w_k:  24  large_w_k:  30


In [ ]:
from google.colab import drive
drive.mount('/content/drive')